<a href="https://colab.research.google.com/github/MikaTina/Project_ML-SII/blob/main/Dictionary_based_Sentiment_Analysis_on_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Import Common Libraries**



In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf

# **Import Dataset**

In [2]:
URL = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file(fname="aclImdb_v1.tar.gz", 
                                  origin=URL,
                                  untar=True,
                                  cache_dir='.',
                                  cache_subdir='')

84131840/84125825 [==============================] - 2s 0us/step


In [3]:
# The shutil module offers a number of high-level 
# operations on files and collections of files.
import os
import shutil
# Create main directory path ("/aclImdb")
main_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
# Create sub directory path ("/aclImdb/train")
train_dir = os.path.join(main_dir, 'train')
# Create sub directory path ("/aclImdb/test")
test_dir = os.path.join(main_dir, 'test')
# Remove unsup folder since this is a supervised learning task
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)
# View the final train and test folder
print(os.listdir(train_dir))
print(os.listdir(test_dir))

['urls_unsup.txt', 'urls_neg.txt', 'neg', 'urls_pos.txt', 'pos', 'labeledBow.feat', 'unsupBow.feat']
['urls_neg.txt', 'neg', 'urls_pos.txt', 'pos', 'labeledBow.feat']


# **Load data from directories**

In [4]:
dir_train_pos = os.listdir(train_dir + "/pos")
dir_train_neg = os.listdir(train_dir + "/neg")
dir_test_pos = os.listdir(test_dir + "/pos")
dir_test_neg = os.listdir(test_dir + "/neg")

In [5]:
data = []
pos_train_dir = os.path.join(train_dir,'pos')
neg_train_dir = os.path.join(train_dir,'neg')
pos_test_dir = os.path.join(test_dir,'pos')
neg_test_dir = os.path.join(test_dir,'neg')

for element in dir_train_pos:
  input_file = open(pos_train_dir + "/"+ element, "r")
  line = input_file.readline()
  data.append([line, "Positive"])
  input_file.close()

for element in dir_train_neg:
  input_file = open(neg_train_dir + "/"+ element, "r")
  line = input_file.readline()
  data.append([line, "Negative"])
  input_file.close()

for element in dir_test_pos:
  input_file = open(pos_test_dir + "/"+ element, "r")
  line = input_file.readline()
  data.append([line, "Positive"])
  input_file.close()

for element in dir_test_neg:
  input_file = open(neg_test_dir + "/"+ element, "r")
  line = input_file.readline()
  data.append([line, "Negative"])
  input_file.close()

# **DataFrame Creation**

In [6]:
df = pd.DataFrame(data, columns=['review', 'sentiment'])

In [7]:
df

,review,sentiment
0,A deplorable social condition triggers off the...,Positive
1,"I loved this movie! It's truly bizarre, extrem...",Positive
2,I would not have known about this film if not ...,Positive
3,"Ok, so it borrows a little from ""It's a Wonder...",Positive
4,"Despite what the title may imply, ""Pigs Is Pig...",Positive
...,...,...
49995,who reads these comments may think we may have...,Negative
49996,This film is one of the worst adaptations of P...,Negative
49997,"Supposedly, a movie about a magazine sending j...",Negative
49998,Tom & Jerry are visiting Africa and disguise t...,Negative


# **First method: pysentiment2**
Pysentiment is a library used for dictionary-based sentiment analysis. 

In [8]:
!pip install pysentiment2

     |████████████████████████████████| 1.9MB 5.1MB/s 


In [9]:
import pysentiment2 as ps

Two dictionaries are provided in the library: Harvard IV-4 and Loughran and McDonald Financial Sentiment Dictionaries, which are sentiment dictionaries for general and financial sentiment analysis. In this work the dictionary chosen is the general one, so Harvard IV-4.

In [11]:
hiv4 = ps.HIV4()

The following variables are used to examine the results of the sentiment analysis: the first one is used to compute the percentage of accuracy, while the others are used to compute precision, recall and f1 score.

In [12]:
# variables initialization
true_pos_ps=0     # number of positive reviews classified as positive
true_neg_ps=0     # number of negative reviews classified as negative
false_pos_ps=0    # number of negative reviews classified as positive
false_neg_ps=0    # number of positive reviews classified as negative

The score computed using this library has 4 components:

* Negative: number of negative tokens
* Positive: number of positive tokens
* Polarity: this is computed as (Pos-Neg)/(Pos+Neg) where Pos and Neg are respectively the number of positive and negative tokens, and it is used to classify the review
* Subjectivity: this is computed as (Pos+Neg)/count(*)

In this work we only use the 'Polarity' component of the score, and we have decided to classify the review as positive if this component is greater than or equal to 0, and as negative if it is lower than 0.

In [ ]:
for row in df.itertuples():
  # tokenization of the review
  tokens = hiv4.tokenize(row.review)
  # score computation
  score = hiv4.get_score(tokens)
  # increment of the involved variables:
  # if the review was labeled as positive and classified as positive increment true_pos_ps
  if ((score['Polarity']>0 or score['Polarity']==0) and row.sentiment=='Positive'):
    true_pos_ps=true_pos_ps+1
  # if the review was labeled as negative and classified as negative increment true_neg_ps
  elif (score['Polarity']<0 and row.sentiment=='Negative'):
    true_neg_ps=true_neg_ps+1
  # if the review was labeled as negative and classified as positive increment false_pos_ps
  elif ((score['Polarity']>0 or score['Polarity']==0) and row.sentiment=='Negative'):
    false_pos_ps=false_pos_ps+1
  # if the review was labeled as positive and classified as negative increment false_neg_ps
  elif (score['Polarity']<0 and row.sentiment=='Positive'):
    false_neg_ps=false_neg_ps+1
  # print to check the status of the computation
  if (row.Index%500==0):
    print(row.Index)

## Results analysis

In [14]:
# ACCURACY
accuracy_ps = (true_pos_ps+true_neg_ps)/(true_pos_ps+true_neg_ps+false_pos_ps+false_neg_ps)
print('The accuracy percentage is ' + str(round(accuracy_ps*100,3)) + '%')

The accuracy percentage is 63.612%


In [15]:
# PRECISION
precision_ps = true_pos_ps/(true_pos_ps+false_pos_ps)
print('The precision is ' + str(round(precision_ps,3)))

The precision is 0.596


In [16]:
# RECALL
recall_ps = true_pos_ps/(true_pos_ps+false_neg_ps)
print('The recall is ' + str(round(recall_ps,3)))

The recall is 0.845


In [17]:
# F1-SCORE
f_score_ps=(2*precision_ps*recall_ps)/(precision_ps+recall_ps)
print('The f1-score is ' + str(round(f_score_ps,3)))

The f1-score is 0.699


# **Second method: NLTK and Vader**
NLTK (Natural Language Tool Kit) is a library for NLP, while Vader (Valence Aware Dictionary e sEntiment Reasoner) is instead a rule-based method for
sentiment analysis, tuned to sentiments from social media.


In [18]:
!pip install nltk

In [19]:
import nltk

In [20]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [21]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
vader = SentimentIntensityAnalyzer()

/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [22]:
# variables initialization
true_pos_nltk=0     # number of positive reviews classified as positive
true_neg_nltk=0     # number of negative reviews classified as negative
false_pos_nltk=0    # number of negative reviews classified as positive
false_neg_nltk=0    # number of positive reviews classified as negative

The score computed using this library has 4 components:

* pos: proportion of the text that falls in the positive category
* neu: proportion of the text that falls in the neutral category
* neg: proportion of the text that falls in the negative category
* compound: metric that calculates the sum of the ratings (one for each word) and normalizes it between -1 (extremely negative) and +1 (extremely positive)

In this work we only use the 'compound' component of the score, and we have decided to classify the review as positive if this component is greater than or equal to 0, and as negative if it is lower than 0.

In [ ]:
for row in df.itertuples():
  # score computation
  score = vader.polarity_scores(row.review)
  # increment of the involved variables:
  # if the review was labeled as positive and classified as positive increment true_pos_ps
  if ((score['compound']>0 or score['compound']==0) and row.sentiment=='Positive'):
    true_pos_nltk=true_pos_nltk+1
  # if the review was labeled as negative and classified as negative increment true_neg_ps
  elif (score['compound']<0 and row.sentiment=='Negative'):
    true_neg_nltk=true_neg_nltk+1
  # if the review was labeled as negative and classified as positive increment false_pos_ps
  elif ((score['compound']>0 or score['compound']==0) and row.sentiment=='Negative'):
    false_pos_nltk=false_pos_nltk+1
  # if the review was labeled as positive and classified as negative increment false_neg_ps
  elif (score['compound']<0 and row.sentiment=='Positive'):
    false_neg_nltk=false_neg_nltk+1
  # print to check the status of the computation
  if row.Index%500==0:
    print(row.Index)

## Results analysis

In [24]:
# ACCURACY
accuracy_nltk = (true_pos_nltk+true_neg_nltk)/(true_pos_nltk+true_neg_nltk+false_pos_nltk+false_neg_nltk)
print('The accuracy percentage is ' + str(round(accuracy_nltk*100,3)) + ' %')

The accuracy percentage is 69.532 %


In [25]:
# PRECISION
precision_nltk = true_pos_nltk/(true_pos_nltk+false_pos_nltk)
print('The precision is ' + str(round(precision_nltk,3)))

The precision is 0.648


In [26]:
# RECALL
recall_nltk = true_pos_nltk/(true_pos_nltk+false_neg_nltk)
print('The recall is ' + str(round(recall_nltk,3)))

The recall is 0.856


In [27]:
# F1-SCORE
f_score_nltk=(2*precision_nltk*recall_nltk)/(precision_nltk+recall_nltk)
print('The f1-score is ' + str(round(f_score_nltk,3)))

The f1-score is 0.738
